In [1]:
# TESTING FOR NSE
# ***************

# Following to be uncommented on first run ...

from helper import get_connected
from ib_insync import *
util.startLoop()
ib = get_connected('nse', 'live') # close the connection and delete it final function

fspath = '../data/nse/'

util.logToFile(fspath+'_errors.log')  # create log file
with open(fspath+'_errors.log', 'w'): # clear the previous log
    pass

In [ ]:
%%time
# WEEKEND GENERATION PROCESS - UNDERLYINGS FIRST
# ______________________________________________

# testing pickle nses function

from nse_func import p_nses
df = p_nses(ib)
df

In [ ]:
%%time
# WEEKEND GENERATION PROCESS - OPTIONS SECOND
# ______________________________________________

# testing pickle nse options function

import pandas as pd
from os import listdir

from nse_func import p_nseopts

keep_pickles = True   # keep already pickled symbols

fspath = '../data/nse/'

df = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()

if keep_pickles:
    fs = listdir(fspath)
    optsList = [f[:-4] for f in fs if f[-3:] == 'pkl']
    dfr = df[~df.index.isin(optsList)] # remaining dfs (without existing pickles)
    [p_nseopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m) for u, p, z, m in zip(dfr.contract, dfr.undPrice, dfr.lot, dfr.margin)]
else:
    [p_nseopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m) for u, p, z, m in zip(df.contract, df.undPrice, df.lot, df.margin)]

In [ ]:
# EVERYDAY UPDATE PROCESS (FIRST PART)
# ___________________________________

# testing nse underlying update function

import pandas as pd
fspath = '../data/nse/'
dfu = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()
dfu

In [ ]:
dfu.loc[:, ['margin', 'undPrice']] = 0 # zerorize margin and undPrice to test
dfu

In [ ]:
%%time

from nse_func import upd_nses
upd_nses(ib, dfu)

In [ ]:
%%time

# EVERYDAY UPDATE PROCESS (SECOND PART - OPT PRICE)
# ________________________________________________

# Option price update
# Note: If the DataFrame given is a 'target' one, it will be every minute

import pandas as pd
fspath = '../data/nse/'

from os import listdir

from helper import upd_opt

blk = 50
fs = listdir(fspath)

# collect the options
optsList = [f for f in fs if f[-3:] == 'pkl']
dfopts = pd.concat([pd.read_pickle(fspath+f) 
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)

# all options with price updated
df = upd_opt(ib, dfopts)

In [ ]:
df.head()

In [ ]:
%%time
# EVERYDAY UPDATE PROCESS (THIRD PART - REMAINING QUANTITIES PRICE)
# _________________________________________________________________

from nse_func import remqty_nse
dfrq = remqty_nse(ib) # remaining quantities df

In [ ]:
# MANAGE NSE (FIRST PART - GENERATE TARGETS WITH expPrice and pickle)
# ___________________________________________________________________

from nse_func import target_nse
dft = target_nse(ib)

In [ ]:
# MANAGE NSE (SECOND PART - GENERATE expPrice FOR TARGETS AND PICKLE)
# ___________________________________________________________________

# TO COMPLETE...
#   - cancel all openOrders
#   - prepare and place harvestOrders - based on option return curve
#   - refreshes the prices of the target_nse
#   - places the new orders

In [ ]:
import pandas as pd
fspath = '../data/nse/'
df_trades = pd.concat([pd.DataFrame(trades), util.df(trades)], axis=1)
df_trades.columns=[['trade']+list(df_trades)[1:]]
df_trades.to_pickle(fspath+'_tradesDone.pickle')

In [ ]:
# update the option prices
df3 = upd_opt(ib, df2)
df3 = grp_opts(df3) # group the options

In [2]:
import pandas as pd
fspath = '../data/nse/'

dftrades = pd.read_pickle(fspath+'_tradesDone.pickle')

In [17]:
dftrades.contract.iloc[0][0]

Contract(secType='OPT', conId=352132756, symbol='ACC', lastTradeDateOrContractMonth='20190425', strike=1200.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ACC19APR1200PE', tradingClass='ACC')

In [19]:
dft = pd.read_pickle(fspath+'_targetopts.pickle')

In [3]:
from ib_insync import *
util.startLoop()

from helper import *
from nse_func import *

if __name__ == '__main__':
    with get_connected('nse', 'live') as ib:
        x = ib.accountSummary()

x

Peer closed connection
clientId 1 already in use?
API connection failed: TimeoutError()


[AccountValue(account='U9329809', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='Cushion', value='0.983012', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='LookAheadNextChange', value='1556682300', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='AccruedCash', value='0.00', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='AvailableFunds', value='22749860.65', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='BuyingPower', value='45499721.31', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='EquityWithLoanValue', value='23243436.60', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='ExcessLiquidity', value='22848755.34', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='FullAvailableFunds', value='22749860.65', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='FullExcessL